In [1]:
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [3]:
raw_data.head()

,laser0,laser1,laser2,laser3,laser4,laser5,laser6,laser7,laser8,laser9,...,laser352,laser353,laser354,laser355,laser356,laser357,laser358,laser359,speed,steering_angle
0,0.227497,0.241333,0.221897,0.244679,0.220947,0.246013,0.231171,0.230307,0.242182,0.231787,...,0.226018,0.208932,0.252255,0.213605,0.241162,0.232063,0.240651,0.218328,NaN,NaN
1,0.227497,0.241333,0.221897,0.244679,0.220947,0.246013,0.231171,0.230307,0.242182,0.231787,...,0.226018,0.208932,0.252255,0.213605,0.241162,0.232063,0.240651,0.218328,NaN,NaN
2,0.258305,0.245077,0.234763,0.248366,0.213277,0.219929,0.225142,0.219998,0.241513,0.225413,...,0.235699,0.249670,0.231843,0.242016,0.245789,0.243913,0.223692,0.230089,NaN,NaN
3,0.240020,0.231830,0.231338,0.207150,0.239114,0.242090,0.231634,0.239512,0.241558,0.249161,...,0.233690,0.249073,0.239164,0.256052,0.230149,0.232954,0.233925,0.235331,NaN,NaN
4,0.250104,0.225447,0.241950,0.230710,0.236610,0.234459,0.226154,0.241565,0.260103,0.232317,...,0.244299,0.225548,0.251118,0.264531,0.228027,0.231908,0.239162,0.247615,NaN,NaN


In [10]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Concatenate, Input, ZeroPadding2D, Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation

In [1]:
data_start = 0
data_stop = 10
data_part = raw_data[data_start:data_stop]
speed = data_part["speed"].values
steering_angle = data_part["steering_angle"].values
output = np.dstack((steering_angle, speed))[0]

img_input = []
h = 240
w = 320
c = 3
for i in range(data_start+1, data_stop+1):
    img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", c)
    img = cv2.resize(img, (w, h))
    img_input.append(img)
img_input = np.asarray(img_input).astype(np.float32) / 255.0
print("img input shape:",img_input.shape)

X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)
if len(X_train.shape) < 4:  # We do this because when image is grey cv2 not add channel num to shape but keras want to see 1 as channel
    X_train.resize(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_validation.resize(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
del img_input
del steering_angle
del output
del speed
print("1st part x train shape:",X_train.shape)
print("1st part y train shape:",y_train.shape)

NameError: name 'raw_data' is not defined

In [12]:
from keras.preprocessing.image import ImageDataGenerator

dataGenerator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=5,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=False,
        vertical_flip=False)
dataGenerator.fit(X_train)

In [13]:
model = Sequential()

model.add(ZeroPadding2D((1,1),input_shape = (h, w, c)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation = "softmax"))

model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

/home/ubunturos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/home/ubunturos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/home/ubunturos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
/home/ubunturos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
/home/ubunturos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")

In [ ]:
hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=8, epochs=7)
#hs = model.fit_generator(dataGenerator.flow(X_train, y_train, batch_size=64), epochs=3, validation_data=[X_validation, y_validation], steps_per_epoch=X_train.shape[0] // 64)

Instructions for updating:
Use tf.cast instead.


In [ ]:
data_start = 10
data_stop = 20
for part in range(1,int(len(raw_data)/10)+1):
    if data_stop > len(raw_data):
        data_stop = len(raw_data)
    if data_stop <= data_start:
        break
    data_part = raw_data[data_start:data_stop]
    speed = data_part["speed"].values
    steering_angle = data_part["steering_angle"].values
    output = np.dstack((steering_angle, speed))[0]
    
    img_input = []
    h = 240
    w = 320
    c = 3
    for i in range(data_start+1, data_stop+1):
        img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", c)
        img = cv2.resize(img, (w, h))
        img_input.append(img)
    img_input = np.asarray(img_input).astype(np.float32) / 255.0
    print("img input shape:",img_input.shape)
    
    X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)
    dataGenerator.fit(X_train)
    del img_input
    del steering_angle
    del output
    del speed
    print(part+1,"th part x train shape:",X_train.shape)
    print(part+1,"th part y train shape:",y_train.shape)
    hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=3)
    #hs = model.fit_generator(dataGenerator.flow(X_train, y_train, batch_size=64), epochs=3, validation_data=[X_validation, y_validation], steps_per_epoch=X_train.shape[0] // 64)
    data_start = data_start + 10
    data_stop = data_stop + 10

In [ ]:
import plotly
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def trainingResultsGraph(model, modelCode):
    trace0 = go.Scatter(
        x = model.epoch,
        y = model.history['loss'],
        mode = 'lines',
        name = 'loss',
        line=dict(color='aquamarine')
    )

    trace1 = go.Scatter(
        x = model.epoch,
        y = model.history['val_loss'],
        mode = 'lines',
        name = 'val_loss',
        line=dict(color='darkred', dash='dash')
    )

    trace2 = go.Scatter(
        x = model.epoch,
        y = model.history['acc'],
        mode = 'lines',
        name = 'acc',
        line=dict(color='violet')
    )

    trace3 = go.Scatter(
        x = model.epoch,
        y = model.history['val_acc'],
        mode = 'lines',
        name = 'val_acc',
        line=dict(color='aqua', dash='dash')
    )

    updatemenus = list([
        dict(type="buttons",
             active=-1,
             buttons=list([
                dict(label = 'Acc Graph',
                     method = 'update',
                     args = [{'visible': [False, False, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation accuracy'}]),
                dict(label = 'Loss Graph',
                     method = 'update',
                     args = [{'visible': [True, True, False, False]},
                             {'title': 'Trained Model'+modelCode+' training and validation loss'}]),
                dict(label = 'Both',
                     method = 'update',
                     args = [{'visible': [True, True, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation values'}])
            ]),
        )
    ])

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title='Trained Model'+modelCode+' training and validation values',
                  xaxis = dict(title = 'Epochs'),
                  updatemenus=updatemenus)

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='lossGraph')
    
trainingResultsGraph(hs, "1")

In [ ]:
try:
    import rospkg
    import rospy

    rospack = rospkg.RosPack()
    package_path = rospack.get_path('angelshark_deeplearning')
    
except ImportError:
    package_path = home_dir + "/Desktop"
    
    import os
    if not os.path.exists(package_path + "/model"):
        os.makedirs(package_path + "/model")
    
    print("Please move them to deeplearning package model directory to angelshark_deeplearning/model")
    
print("Model saved to ",str(package_path)," dir!!!")

In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
model.save_weights(package_path + '/model/model.h5')
model_json = model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

In [ ]:
# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()